In [1]:
from git_actions import GitActions
from prompt import ActionsPromptTemplate
from tools import TOOLS

tool_names = [tool.name for tool in TOOLS]
tools_prompt = "\n".join([f"{tool.name}: {tool.description}" for tool in TOOLS])

actions_template = ActionsPromptTemplate(input_variables=["input", "intermediate_steps"])

In [2]:
from langchain import LLMChain
from output_parser import ActionsOutputParser
from langchain.agents import LLMSingleActionAgent

# LLM chain consisting of the LLM and a prompt
from const import OPEN_AI_API_KEY
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, temperature=0)
llm_chain = LLMChain(llm=llm, prompt=actions_template)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    stop=["Observation: "],
    output_parser=ActionsOutputParser(),
    allowed_tools=tool_names
)

In [3]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=TOOLS, verbose=True)

In [4]:
# Easy case: input as in examples
agent_executor.run(
    input="create branch hot_fixes",
    tool_names=tool_names,
    tools=tools_prompt,
    actions=GitActions.all_actions()
)



> Entering new AgentExecutor chain...
Thought: I need to know the current commit from which to create a branch
Action: get_commit_name

Observation: test_current_commit
I know the final answer
Final_Answer:
NewBranch[hot_fixes, test_current_commit, True]

> Finished chain.


'Creating new branch hot_fixes from commit test_current_commit \n'

In [5]:
# A little bit of changes
agent_executor.run(
    input="save changes with comment 'refactoring'",
    tool_names=tool_names,
    tools=tools_prompt,
    actions=GitActions.all_actions()
)



> Entering new AgentExecutor chain...
Thought: I need to know what files have been changed
Action: get_list_of_changed_files

Observation: [changed_file1, changed_file2, changed_file3]
I need to commit the changes with the given comment
Final_Answer:
Commit[[changed_file1, changed_file2, changed_file3], refactoring, False]

> Finished chain.


'Committing [changed_file1 with message changed_file2, isCheckout=changed_file3] \n'

In [6]:
# Harder:
agent_executor.run(
    input="amend last commit",
    tool_names=tool_names,
    tools=tools_prompt,
    actions=GitActions.all_actions()
)
# There is no tool to get e message of the last commit



> Entering new AgentExecutor chain...
Thought: I need to know what files were changed in the last commit
Action: get_list_of_changed_files

Observation: [changed_file1, changed_file2, changed_file3]
I know the final answer
Final_Answer:
Commit[[changed_file1, changed_file2, changed_file3], ?, True]

> Finished chain.


'Committing [changed_file1 with message changed_file2, isCheckout=changed_file3] \n'

In [7]:
agent_executor.run(
    input="what's changed in the last commit?",
    tool_names=tool_names,
    tools=tools_prompt,
    actions=GitActions.all_actions()
)
# Sometimes it generates wrong semantics: ShowDiff[Item] should have parameter of a file, not a commit



> Entering new AgentExecutor chain...
Thought: I need to know the current commit to show the changes made in the last commit.
Action: get_commit_name

Observation: test_current_commit
I know the final answer
Final_Answer:
ShowDiff[test_current_commit]

> Finished chain.


'Showing diff of test_current_commit \n'

In [8]:
# Sequence of actions
agent_executor.run(
    input="update and push the changes",
    tool_names=tool_names,
    tools=tools_prompt,
    actions=GitActions.all_actions()
)



> Entering new AgentExecutor chain...
Thought: I need to know what changes have been made
Action: get_list_of_changed_files

Observation: [changed_file1, changed_file2, changed_file3]
I need to know current remote
Action: get_remote_name


Observation: test_remote
I need to know current branch
Action: get_branch_name


Observation: test_branch
I need to add the changes to the staging area and commit them with a message. Then, I can push the changes to the remote branch.

Action: 
Add[[changed_file1, changed_file2, changed_file3]]
Commit[[changed_file1, changed_file2, changed_file3], "Update files", False]
Push[test_remote, test_branch] 

Final_Answer:
Add[[changed_file1, changed_file2, changed_file3]]
Commit[[changed_file1, changed_file2, changed_file3], "Update files", False]
Push[test_remote, test_branch]

> Finished chain.


'Adding [changed_file1 \nCommitting [changed_file1 with message changed_file2, isCheckout=changed_file3] \nPushing to test_remote / test_branch \n'

In [9]:
# Let's check if it outputs predefined answer 'UNKNOWN' if there is something wrong
agent_executor.run(
    input="how old are you?",
    tool_names=tool_names,
    tools=tools_prompt,
    actions=GitActions.all_actions()
)
# So it does not



> Entering new AgentExecutor chain...


ValueError: Could not parse LLM output: `I'm sorry, I cannot answer that question as it is not related to IDE actions. Please ask a question related to IDE actions.`